# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd 
import re
from sqlalchemy import create_engine
import nltk
nltk.download(['stopwords','punkt','wordnet'])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('DisasterResponseETL', engine)

In [3]:
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [4]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [5]:
Y.head()

,"('related',)","('request',)","('offer',)","('aid_related',)","('medical_help',)","('medical_products',)","('search_and_rescue',)","('security',)","('military',)","('child_alone',)",...,"('aid_centers',)","('other_infrastructure',)","('weather_related',)","('floods',)","('storm',)","('fire',)","('earthquake',)","('cold',)","('other_weather',)","('direct_report',)"
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    text = re.sub('[^a-zA-Z0-9]',' ',text) # Remove characters that not in A-Z, a-z, 0-9
    
    text = text.lower() # Convert all characters to lowercase
    
    words = word_tokenize(text) # Tokenize text into list of words 
    
    enstopwords = stopwords.words("english")
    words = [w for w in words if not w in enstopwords] # Remove stopwords in English
    
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words] # Lemmatize words
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = pipeline = Pipeline([
                   ('vect', CountVectorizer(tokenizer=tokenize)),
                   ('tfidf', TfidfTransformer()),
                   ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
pred = pipeline.predict(X_test)


In [11]:
num_categories = y_test.shape[1]
for index in range(num_categories):
    col = y_test.columns[index]
    print(col)
    print(classification_report(y_test[col], pred[:, index]))
    
    

('related',)
             precision    recall  f1-score   support

          0       0.63      0.46      0.53      1509
          1       0.85      0.91      0.88      5004
          2       0.23      0.39      0.29        41

avg / total       0.79      0.81      0.80      6554

('request',)
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      5419
          1       0.78      0.43      0.56      1135

avg / total       0.87      0.88      0.87      6554

('offer',)
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6531
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      6554

('aid_related',)
             precision    recall  f1-score   support

          0       0.75      0.87      0.80      3812
          1       0.77      0.59      0.67      2742

avg / total       0.76      0.75      0.75      6554

('medical_help',)
             pr

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
        'clf__estimator__n_estimators': [10, 100],
}

cv = GridSearchCV(pipeline, param_grid=parameters) 

In [ ]:
cv.fit(X_train, y_train)


In [ ]:
cv.cv_results_

In [ ]:
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_improved = cv.predict(X_test)
for i in range(Y.shape[1]):
    print(Y.columns[i], ':')
    print(classification_report(y_test.values[:,i], y_pred_improved[:,i]), '_'*50)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# using SVM instead 
pipeline_SVM = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

parameters_SVM = {'vect__min_df': [5],
              'tfidf__use_idf':[True],
              'clf__estimator__kernel': ['poly'], 
              'clf__estimator__degree': [2, 3],
              'clf__estimator__C':[10, 100]}

cv2 = GridSearchCV(pipeline_SVM, param_grid=parameters_SVM, verbose=2)

# Find best parameters
np.random.seed(77)
SVM_model = cv2.fit(X_train, y_train)

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv, open('disaster_response_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.